In [1]:
!apt-get update -qq
!apt-get install openjdk-11-jdk-headless -qq


W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Selecting previously unselected package openjdk-11-jre-headless:amd64.
(Reading database ... 117528 files and directories currently installed.)
Preparing to unpack .../openjdk-11-jre-headless_11.0.29+7-1ubuntu1~22.04_amd64.deb ...
Unpacking openjdk-11-jre-headless:amd64 (11.0.29+7-1ubuntu1~22.04) ...
Selecting previously unselected package openjdk-11-jdk-headless:amd64.
Preparing to unpack .../openjdk-11-jdk-headless_11.0.29+7-1ubuntu1~22.04_amd64.deb ...
Unpacking openjdk-11-jdk-headless:amd64 (11.0.29+7-1ubuntu1~22.04) ...
Setting up openjdk-11-jre-headless:amd64 (11.0.29+7-1ubuntu1~22.04) ...
update-alternatives: using /usr/lib/jvm/java-11-openjdk-amd64/bin/jjs to provide /usr/bin/jjs (jjs) in auto mode
update-alternatives: using /usr/lib/jvm/java-11-openjdk-amd64/bin/rmid to provide /usr/bin/rmid

In [2]:
!pip install -q pyspark==3.4.1


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.5/200.5 kB 14.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dataproc-spark-connect 1.0.1 requires pyspark[connect]~=4.0.0, but you have pyspark 3.4.1 which is incompatible.


In [3]:
!wget -q https://archive.apache.org/dist/spark/spark-3.4.1/spark-3.4.1-bin-hadoop3.tgz
!tar xf spark-3.4.1-bin-hadoop3.tgz


In [4]:
import os

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.4.1-bin-hadoop3"
os.environ["PATH"] += ":/content/spark-3.4.1-bin-hadoop3/bin"


In [5]:
!ls /content/spark-3.4.1-bin-hadoop3/bin | grep spark-submit


spark-submit
spark-submit2.cmd
spark-submit.cmd


In [6]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName("Telecom Churn Prediction") \
    .config("spark.ui.showConsoleProgress", "false") \
    .getOrCreate()

spark


In [7]:
from google.colab import files

uploaded = files.upload()


Saving TelcoCustomerChurn.csv to TelcoCustomerChurn.csv


In [8]:
import os

os.listdir()


['.config',
 'TelcoCustomerChurn.csv',
 'spark-3.4.1-bin-hadoop3',
 'spark-3.4.1-bin-hadoop3.tgz',
 'sample_data']

In [9]:
df = spark.read.csv(
    "TelcoCustomerChurn.csv",
    header=True,
    inferSchema=True
)

df.show(5)
df.printSchema()


+----------+------+-------------+-------+----------+------+------------+----------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------------+----------------+--------------------+--------------+------------+-----+
|customerID|gender|SeniorCitizen|Partner|Dependents|tenure|PhoneService|   MultipleLines|InternetService|OnlineSecurity|OnlineBackup|DeviceProtection|TechSupport|StreamingTV|StreamingMovies|      Contract|PaperlessBilling|       PaymentMethod|MonthlyCharges|TotalCharges|Churn|
+----------+------+-------------+-------+----------+------+------------+----------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------------+----------------+--------------------+--------------+------------+-----+
|7590-VHVEG|Female|            0|    Yes|        No|     1|          No|No phone service|            DSL|            No|         Yes|              No|         No|    

In [10]:
print("Total rows:", df.count())
df.groupBy("Churn").count().show()


Total rows: 7040
+-----+-----+
|Churn|count|
+-----+-----+
|   No| 5171|
|  Yes| 1869|
+-----+-----+



In [11]:
df = spark.read.csv(
    "TelcoCustomerChurn.csv",
    header=True,
    inferSchema=True
)

df.printSchema()
df.show(5)


root
 |-- customerID: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- SeniorCitizen: integer (nullable = true)
 |-- Partner: string (nullable = true)
 |-- Dependents: string (nullable = true)
 |-- tenure: integer (nullable = true)
 |-- PhoneService: string (nullable = true)
 |-- MultipleLines: string (nullable = true)
 |-- InternetService: string (nullable = true)
 |-- OnlineSecurity: string (nullable = true)
 |-- OnlineBackup: string (nullable = true)
 |-- DeviceProtection: string (nullable = true)
 |-- TechSupport: string (nullable = true)
 |-- StreamingTV: string (nullable = true)
 |-- StreamingMovies: string (nullable = true)
 |-- Contract: string (nullable = true)
 |-- PaperlessBilling: string (nullable = true)
 |-- PaymentMethod: string (nullable = true)
 |-- MonthlyCharges: double (nullable = true)
 |-- TotalCharges: string (nullable = true)
 |-- Churn: string (nullable = true)

+----------+------+-------------+-------+----------+------+------------+---------

In [12]:
total_rows = df.count()

churn_distribution = (
    df.groupBy("Churn")
      .count()
      .withColumnRenamed("count", "customers")
)

print("Total records:", total_rows)
churn_distribution.show()


Total records: 7040
+-----+---------+
|Churn|customers|
+-----+---------+
|   No|     5171|
|  Yes|     1869|
+-----+---------+



In [13]:
from pyspark.sql.functions import col, sum as spark_sum

df.select([
    spark_sum(col(c).isNull().cast("int")).alias(c)
    for c in df.columns
]).show()


+----------+------+-------------+-------+----------+------+------------+-------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------+----------------+-------------+--------------+------------+-----+
|customerID|gender|SeniorCitizen|Partner|Dependents|tenure|PhoneService|MultipleLines|InternetService|OnlineSecurity|OnlineBackup|DeviceProtection|TechSupport|StreamingTV|StreamingMovies|Contract|PaperlessBilling|PaymentMethod|MonthlyCharges|TotalCharges|Churn|
+----------+------+-------------+-------+----------+------+------------+-------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------+----------------+-------------+--------------+------------+-----+
|         0|     0|            0|      0|         0|     0|           0|            0|              0|             0|           0|               0|          0|          0|              0|       0|               0| 

In [14]:
from pyspark.sql.functions import col, trim

df.filter(trim(col("TotalCharges")) == "").count()


8

In [15]:
from pyspark.sql.functions import col, trim


In [16]:
df = df.filter(trim(col("TotalCharges")) != "")


In [17]:
df = df.withColumn(
    "TotalCharges",
    col("TotalCharges").cast("double")
)


In [18]:
df = df.dropna(subset=["TotalCharges"])


In [19]:
df = df.drop("customerID")


In [20]:
df = df.withColumn(
    "Churn",
    (col("Churn") == "Yes").cast("int")
)


In [21]:
from pyspark.sql.functions import sum as spark_sum

df.select([
    spark_sum(col(c).isNull().cast("int")).alias(c)
    for c in df.columns
]).show()

print("Final row count:", df.count())


+------+-------------+-------+----------+------+------------+-------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------+----------------+-------------+--------------+------------+-----+
|gender|SeniorCitizen|Partner|Dependents|tenure|PhoneService|MultipleLines|InternetService|OnlineSecurity|OnlineBackup|DeviceProtection|TechSupport|StreamingTV|StreamingMovies|Contract|PaperlessBilling|PaymentMethod|MonthlyCharges|TotalCharges|Churn|
+------+-------------+-------+----------+------+------------+-------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------+----------------+-------------+--------------+------------+-----+
|     0|            0|      0|         0|     0|           0|            0|              0|             0|           0|               0|          0|          0|              0|       0|               0|            0|             0|           0|   

In [24]:
numerical_cols = ["tenure", "MonthlyCharges", "TotalCharges"]


In [25]:
def get_iqr_bounds(df, col_name):
    q1, q3 = df.approxQuantile(col_name, [0.25, 0.75], 0.01)
    iqr = q3 - q1
    lower = q1 - 1.5 * iqr
    upper = q3 + 1.5 * iqr
    return lower, upper


In [26]:
bounds = {}

for col_name in numerical_cols:
    lower, upper = get_iqr_bounds(df, col_name)
    bounds[col_name] = (lower, upper)
    print(f"{col_name} → Lower: {lower:.2f}, Upper: {upper:.2f}")


tenure → Lower: -60.00, Upper: 124.00
MonthlyCharges → Lower: -45.20, Upper: 170.40
TotalCharges → Lower: -4517.42, Upper: 8602.38


In [27]:
from pyspark.sql.functions import when, col


In [28]:
mc_lower, mc_upper = bounds["MonthlyCharges"]

df = df.withColumn(
    "MonthlyCharges",
    when(col("MonthlyCharges") > mc_upper, mc_upper)
    .otherwise(col("MonthlyCharges"))
)


In [29]:
tc_lower, tc_upper = bounds["TotalCharges"]

df = df.withColumn(
    "TotalCharges",
    when(col("TotalCharges") > tc_upper, tc_upper)
    .otherwise(col("TotalCharges"))
)


In [30]:
for col_name in ["MonthlyCharges", "TotalCharges"]:
    lower, upper = bounds[col_name]
    remaining = df.filter(col(col_name) > upper).count()
    print(f"{col_name} remaining upper outliers: {remaining}")


MonthlyCharges remaining upper outliers: 0
TotalCharges remaining upper outliers: 0


In [31]:
categorical_cols = [
    "gender", "Partner", "Dependents", "PhoneService",
    "MultipleLines", "InternetService", "OnlineSecurity",
    "OnlineBackup", "DeviceProtection", "TechSupport",
    "StreamingTV", "StreamingMovies", "Contract",
    "PaperlessBilling", "PaymentMethod"
]

numerical_cols = ["tenure", "MonthlyCharges", "TotalCharges"]


In [32]:
from pyspark.ml.feature import StringIndexer

indexer = StringIndexer(
    inputCols=categorical_cols,
    outputCols=[c + "_idx" for c in categorical_cols],
    handleInvalid="keep"
)

df = indexer.fit(df).transform(df)


In [33]:
df

DataFrame[gender: string, SeniorCitizen: int, Partner: string, Dependents: string, tenure: int, PhoneService: string, MultipleLines: string, InternetService: string, OnlineSecurity: string, OnlineBackup: string, DeviceProtection: string, TechSupport: string, StreamingTV: string, StreamingMovies: string, Contract: string, PaperlessBilling: string, PaymentMethod: string, MonthlyCharges: double, TotalCharges: double, Churn: int, gender_idx: double, Partner_idx: double, Dependents_idx: double, PhoneService_idx: double, MultipleLines_idx: double, InternetService_idx: double, OnlineSecurity_idx: double, OnlineBackup_idx: double, DeviceProtection_idx: double, TechSupport_idx: double, StreamingTV_idx: double, StreamingMovies_idx: double, Contract_idx: double, PaperlessBilling_idx: double, PaymentMethod_idx: double]

In [34]:
df.select(
    "Contract", "Contract_idx",
    "PaymentMethod", "PaymentMethod_idx"
).show(5)


+--------------+------------+--------------------+-----------------+
|      Contract|Contract_idx|       PaymentMethod|PaymentMethod_idx|
+--------------+------------+--------------------+-----------------+
|Month-to-month|         0.0|    Electronic check|              0.0|
|      One year|         2.0|        Mailed check|              1.0|
|Month-to-month|         0.0|        Mailed check|              1.0|
|      One year|         2.0|Bank transfer (au...|              2.0|
|Month-to-month|         0.0|    Electronic check|              0.0|
+--------------+------------+--------------------+-----------------+
only showing top 5 rows



In [35]:
df.select(
    "Contract", "Contract_idx",
    "PaymentMethod", "PaymentMethod_idx"
).show(5)


+--------------+------------+--------------------+-----------------+
|      Contract|Contract_idx|       PaymentMethod|PaymentMethod_idx|
+--------------+------------+--------------------+-----------------+
|Month-to-month|         0.0|    Electronic check|              0.0|
|      One year|         2.0|        Mailed check|              1.0|
|Month-to-month|         0.0|        Mailed check|              1.0|
|      One year|         2.0|Bank transfer (au...|              2.0|
|Month-to-month|         0.0|    Electronic check|              0.0|
+--------------+------------+--------------------+-----------------+
only showing top 5 rows



In [36]:
numerical_cols = ["tenure", "MonthlyCharges", "TotalCharges"]


In [37]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(
    inputCols=numerical_cols + [c + "_idx" for c in categorical_cols],
    outputCol="features"
)

df = assembler.transform(df)


In [38]:
df.select("features", "Churn").show(5, truncate=False)


+-------------------------------------------------------------------------------+-----+
|features                                                                       |Churn|
+-------------------------------------------------------------------------------+-----+
|(18,[0,1,2,3,4,6,7,8,10],[1.0,29.85,29.85,1.0,1.0,1.0,2.0,1.0,1.0])            |0    |
|(18,[0,1,2,8,9,11,15,16,17],[34.0,56.95,1889.5,1.0,1.0,1.0,2.0,1.0,1.0])       |0    |
|(18,[0,1,2,8,9,10,17],[2.0,53.85,108.15,1.0,1.0,1.0,1.0])                      |1    |
|[45.0,42.3,1840.75,0.0,0.0,0.0,1.0,2.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,2.0,1.0,2.0]|0    |
|(18,[0,1,2,3],[2.0,70.7,151.65,1.0])                                           |1    |
+-------------------------------------------------------------------------------+-----+
only showing top 5 rows



In [39]:
train_df, test_df = df.randomSplit([0.8, 0.2], seed=42)

print("Training rows:", train_df.count())
print("Testing rows:", test_df.count())


Training rows: 5690
Testing rows: 1342


In [41]:
from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression(
    featuresCol="features",
    labelCol="Churn",
    maxIter=30,
    regParam=0.01
)

lr_model = lr.fit(train_df)


In [42]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

evaluator = BinaryClassificationEvaluator(
    labelCol="Churn",
    metricName="areaUnderROC"
)

lr_predictions = lr_model.transform(test_df)
lr_auc = evaluator.evaluate(lr_predictions)

print("Logistic Regression AUC:", lr_auc)


Logistic Regression AUC: 0.8478711948099766


In [43]:
from pyspark.ml.classification import RandomForestClassifier

rf = RandomForestClassifier(
    featuresCol="features",
    labelCol="Churn",
    numTrees=120,
    maxDepth=10,
    seed=42
)

rf_model = rf.fit(train_df)


In [44]:
rf_predictions = rf_model.transform(test_df)
rf_auc = evaluator.evaluate(rf_predictions)

print("Random Forest AUC:", rf_auc)


Random Forest AUC: 0.853652486305553


In [45]:
rf_preds = rf_predictions.select("Churn", "prediction")
rf_preds.show(5)


+-----+----------+
|Churn|prediction|
+-----+----------+
|    0|       1.0|
|    1|       1.0|
|    0|       0.0|
|    0|       0.0|
|    1|       1.0|
+-----+----------+
only showing top 5 rows



In [46]:
rf_preds.groupBy("Churn", "prediction").count().show()


+-----+----------+-----+
|Churn|prediction|count|
+-----+----------+-----+
|    1|       0.0|  174|
|    0|       0.0|  906|
|    1|       1.0|  169|
|    0|       1.0|   93|
+-----+----------+-----+



In [48]:
accuracy = (906 + 169) / (906 + 93 + 174 + 169)
accuracy


0.801043219076006

In [49]:
precision = 169 / (169 + 93)
precision


0.6450381679389313

In [ ]:
recall = 169 / (169 + 174)
recall


0.49271137026239065